# Disease Prediction based on Symtoms

In [1]:
#Importing Libraries
!pip install python-Levenshtein
import Levenshtein

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tkinter import *
import numpy as np
import pandas as pd
import os

In [2]:
#List of the symptoms is listed here in list l1.

l1=['back pain','constipation','abdominal pain','diarrhoea','mild fever','yellow urine',
    'yellowing of eyes','acute liver failure','fluid overload','swelling of stomach',
    'swelled lymph nodes','malaise','blurred and distorted vision','phlegm','throat irritation',
    'redness of eyes','sinus pressure','runny nose','congestion','chest pain','weakness in limbs',
    'fast heart rate','pain during bowel movements','pain in anal region','bloody stool',
    'irritation in anus','neck pain','dizziness','cramps','bruising','obesity','swollen legs',
    'swollen blood vessels','puffy face and eyes','enlarged thyroid','brittle nails',
    'swollen extremeties','excessive hunger','extra marital contacts','drying and tingling lips',
    'slurred speech','knee pain','hip joint pain','muscle weakness','stiff neck','swelling joints',
    'movement stiffness','spinning movements','loss of balance','unsteadiness',
    'weakness of one body side','loss of smell','bladder discomfort','foul smell of urine',
    'continuous feel of urine','passage of gases','internal itching','toxic look (typhos)',
    'depression','irritability','muscle pain','altered sensorium','red spots over body','belly pain',
    'abnormal menstruation','dischromic  patches','watering from eyes','increased appetite','polyuria','family history','mucoid sputum',
    'rusty sputum','lack of concentration','visual disturbances','receiving blood transfusion',
    'receiving unsterile injections','coma','stomach bleeding','distention of abdomen',
    'history of alcohol consumption','fluid overload','blood in sputum','prominent veins on calf',
    'palpitations','painful walking','pus filled pimples','blackheads','scurring','skin peeling',
    'silver like dusting','small dents in nails','inflammatory nails','blister','red sore around nose',
    'yellow crust ooze'
]

In [3]:
#List of Diseases is listed in list disease.

disease=['Fungal infection', 'Allergy', 'GERD', 'Chronic cholestasis',
       'Drug Reaction', 'Peptic ulcer diseae', 'AIDS', 'Diabetes ',
       'Gastroenteritis', 'Bronchial Asthma', 'Hypertension ', 'Migraine',
       'Cervical spondylosis', 'Paralysis (brain hemorrhage)', 'Jaundice',
       'Malaria', 'Chicken pox', 'Dengue', 'Typhoid', 'hepatitis A',
       'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E',
       'Alcoholic hepatitis', 'Tuberculosis', 'Common Cold', 'Pneumonia',
       'Dimorphic hemmorhoids(piles)', 'Heart attack', 'Varicose veins',
       'Hypothyroidism', 'Hyperthyroidism', 'Hypoglycemia',
       'Osteoarthristis', 'Arthritis',
       '(vertigo) Paroymsal  Positional Vertigo', 'Acne',
       'Urinary tract infection', 'Psoriasis', 'Impetigo']

#disease = [df['prognosis'].unique()]
#print(disease)

In [4]:
l2=[]
for i in range(0,len(l1)):
    l2.append(0)
print(l2)

In [5]:
#Reading the training .csv file
df=pd.read_csv("training.csv")
DF= pd.read_csv('training.csv', index_col='prognosis')
#Replace the values in the imported file by pandas by the inbuilt function replace in pandas.

df.replace({'prognosis':{'Fungal infection':0,'Allergy':1,'GERD':2,'Chronic cholestasis':3,'Drug Reaction':4,
    'Peptic ulcer diseae':5,'AIDS':6,'Diabetes ':7,'Gastroenteritis':8,'Bronchial Asthma':9,'Hypertension ':10,
    'Migraine':11,'Cervical spondylosis':12,
    'Paralysis (brain hemorrhage)':13,'Jaundice':14,'Malaria':15,'Chicken pox':16,'Dengue':17,'Typhoid':18,'hepatitis A':19,
    'Hepatitis B':20,'Hepatitis C':21,'Hepatitis D':22,'Hepatitis E':23,'Alcoholic hepatitis':24,'Tuberculosis':25,
    'Common Cold':26,'Pneumonia':27,'Dimorphic hemmorhoids(piles)':28,'Heart attack':29,'Varicose veins':30,'Hypothyroidism':31,
    'Hyperthyroidism':32,'Hypoglycemia':33,'Osteoarthristis':34,'Arthritis':35,
    '(vertigo) Paroymsal  Positional Vertigo':36,'Acne':37,'Urinary tract infection':38,'Psoriasis':39,
    'Impetigo':40}},inplace=True)
#df.head()
DF.head()

In [6]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df1, nGraphShown, nGraphPerRow):
    nunique = df1.nunique()
    df1 = df1[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df1.shape
    columnNames = list(df1)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [7]:
# Scatter and density plots
def plotScatterMatrix(df1, plotSize, textSize):
    df1 = df1.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df1 = df1.dropna('columns')
    df1 = df1[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df1 = df1[columnNames]
    ax = pd.plotting.scatter_matrix(df1, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df1.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()


In [8]:
#plotPerColumnDistribution(df, 10, 5)

In [9]:
#plotScatterMatrix(df, 20, 10)

In [10]:
X= df[l1]
y = df[["prognosis"]]
np.ravel(y)
print(X)

In [11]:
print(y)

In [12]:
#Reading the  testing.csv file
tr=pd.read_csv("testing.csv")

#Using inbuilt function replace in pandas for replacing the values

tr.replace({'prognosis':{'Fungal infection':0,'Allergy':1,'GERD':2,'Chronic cholestasis':3,'Drug Reaction':4,
    'Peptic ulcer diseae':5,'AIDS':6,'Diabetes ':7,'Gastroenteritis':8,'Bronchial Asthma':9,'Hypertension ':10,
    'Migraine':11,'Cervical spondylosis':12,
    'Paralysis (brain hemorrhage)':13,'Jaundice':14,'Malaria':15,'Chicken pox':16,'Dengue':17,'Typhoid':18,'hepatitis A':19,
    'Hepatitis B':20,'Hepatitis C':21,'Hepatitis D':22,'Hepatitis E':23,'Alcoholic hepatitis':24,'Tuberculosis':25,
    'Common Cold':26,'Pneumonia':27,'Dimorphic hemmorhoids(piles)':28,'Heart attack':29,'Varicose veins':30,'Hypothyroidism':31,
    'Hyperthyroidism':32,'Hypoglycemia':33,'Osteoarthristis':34,'Arthritis':35,
    '(vertigo) Paroymsal  Positional Vertigo':36,'Acne':37,'Urinary tract infection':38,'Psoriasis':39,
    'Impetigo':40}},inplace=True)
tr.head()

In [13]:
X_test= tr[l1]
y_test = tr[["prognosis"]]
np.ravel(y_test)
print(X_test)

In [14]:
print(y_test)

# Random Forest Algorithm

In [15]:
def find_closest_match(input_text, word_list):
    highest_ratio = 0
    closest_match = ''

    for word in word_list:
        ratio = Levenshtein.ratio(input_text.lower(), word.lower())
        if ratio > highest_ratio:
            highest_ratio = ratio
            closest_match = word

    return closest_match

In [16]:
import csv
def map_word_to_row(word):
    with open('symptom_Description.csv', 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row
        for row in reader:
            if row[0] == word:
                return row[0], row[1]

    return None, None  # Word not found

In [17]:
root = Tk()
pred2=StringVar()
def randomforest():
        from sklearn.ensemble import RandomForestClassifier
        clf4 = RandomForestClassifier(n_estimators=100)
        clf4 = clf4.fit(X,np.ravel(y))

        # calculating accuracy 
        from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
        y_pred=clf4.predict(X_test)
        print("Random Forest")
        print("Accuracy")
        print(accuracy_score(y_test, y_pred))
        print(accuracy_score(y_test, y_pred,normalize=False))
        print("Confusion matrix")
        conf_matrix=confusion_matrix(y_test,y_pred)
        print(conf_matrix)
    
        psymptoms = [Symptom1.get(),Symptom2.get(),Symptom3.get(),Symptom4.get(),Symptom5.get()]

        for k in range(0,len(l1)):
            for z in psymptoms:
                if(find_closest_match(z, l1)==l1[k]):
                    l2[k]=1

        inputtest = [l2]
        predict = clf4.predict(inputtest)
        predicted=predict[0]
        h='no'
        for a in range(0,len(disease)):
            if(predicted == a):
                h='yes'
                break
        if (h=='yes'):
            pred2.set(" ")
            #getting also the description of the disease
            pred2.set(disease[a] + " - " + map_word_to_row(disease[a])[1])
            
            #formating the pred2 string
            import textwrap
            formatted_output = "\n".join(textwrap.wrap(pred2.get(), width=40))
            pred2.set(formatted_output)
            print(pred2.get())
        else:
            pred2.set(" ")
            pred2.set("Not Found")

# Building Graphical User Interface

In [18]:
#Tk class is used to create a root window
root.configure(background='Ivory')
root.title('Smart Disease Predictor System')
root.resizable(0,0)

In [19]:
Symptom1 = StringVar()
Symptom1.set("")

Symptom2 = StringVar()
Symptom2.set("")

Symptom3 = StringVar()
Symptom3.set("")

Symptom4 = StringVar()
Symptom4.set("")

Symptom5 = StringVar()
Symptom5.set("")
Name = StringVar()

In [20]:
#Headings for the GUI written at the top of GUI
w2 = Label(root, justify=LEFT, text="Disease Predictor using Machine Learning", fg="Red", bg="Ivory")
w2.config(font=("Times",30,"bold italic"))
w2.grid(row=1, column=0, columnspan=2, padx=100)

In [21]:

Label(root, text="Symptom 1:").grid(row=8, column=0)
Label(root, text="Symptom 2:").grid(row=9, column=0)
Label(root, text="Symptom 3:").grid(row=10, column=0)
Label(root, text="Symptom 4:").grid(row=11, column=0)
Label(root, text="Symptom 5:").grid(row=12, column=0)

symptom1_entry = Entry(root,textvariable=Symptom1)
symptom1_entry.grid(row=8, column=1)

symptom2_entry = Entry(root,textvariable=Symptom2)
symptom2_entry.grid(row=9, column=1)

symptom3_entry = Entry(root,textvariable=Symptom3)
symptom3_entry.grid(row=10, column=1)

symptom4_entry = Entry(root,textvariable=Symptom4)
symptom4_entry.grid(row=11, column=1)

symptom5_entry = Entry(root,textvariable=Symptom5)
symptom5_entry.grid(row=12, column=1)


In [22]:
#Buttons for predicting the disease using different algorithms

rnf = Button(root, text="Predict", command=randomforest,bg="Light green",fg="red")
rnf.config(font=("Times",15,"bold italic"))
rnf.grid(row=16, column=0,padx=10)


In [23]:


t2=Label(root,font=("Times",12,"bold italic"),text="Random Forest",height=10,bg="Purple"
         ,width=40,fg="white",textvariable=pred2,relief="sunken").grid(row=7, column=1, padx=10, pady=10)


In [ ]:
#calling this function because the application is ready to run
root.mainloop()